In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# imports
 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.metrics import balanced_accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomTreesEmbedding, RandomForestClassifier
 
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as pipe_with_sampling
from imblearn.pipeline import make_pipeline

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# downcasting
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')

train, test = reduce_mem_usage(train), reduce_mem_usage(test)

# drop zero-variance features and id
train.drop(columns=['Id', 'Soil_Type7', 'Soil_Type15'], inplace=True)
test.drop(columns=['Id', 'Soil_Type7', 'Soil_Type15'], inplace=True)
train.head()

In [ ]:
train['Aspect_ind'] = train['Aspect'].apply(lambda x: int(x < 0))
train['Aspect'] = np.abs(train['Aspect'])
train['Slope_ind'] = train['Slope'].apply(lambda x: int(x < 0))
train['Aspect'] = np.abs(train['Aspect'])

train['Hillshade_9am_ind'] = train['Hillshade_9am'].apply(lambda x: int(x < 0))
train['Aspect'] = np.abs(train['Aspect'])

train['Hillshade_Noon_ind'] = train['Hillshade_Noon'].apply(lambda x: int(x < 0))
train['Aspect'] = np.abs(train['Aspect'])

train['Hillshade_3pm_ind'] = train['Hillshade_3pm'].apply(lambda x: int(x < 0))
train['Aspect'] = np.abs(train['Aspect'])


test['Aspect_ind'] = test['Aspect'].apply(lambda x: int(x < 0))
test['Aspect'] = np.abs(test['Aspect'])
test['Slope_ind'] = test['Slope'].apply(lambda x: int(x < 0))
test['Aspect'] = np.abs(test['Aspect'])

test['Hillshade_9am_ind'] = test['Hillshade_9am'].apply(lambda x: int(x < 0))
test['Aspect'] = np.abs(test['Aspect'])

test['Hillshade_Noon_ind'] = test['Hillshade_Noon'].apply(lambda x: int(x < 0))
test['Aspect'] = np.abs(test['Aspect'])

test['Hillshade_3pm_ind'] = test['Hillshade_3pm'].apply(lambda x: int(x < 0))
test['Aspect'] = np.abs(test['Aspect'])


train['dist_to_hydro'] = np.abs(train['Horizontal_Distance_To_Hydrology']) + np.abs(train['Vertical_Distance_To_Hydrology'])
test['dist_to_hydro'] = np.abs(test['Horizontal_Distance_To_Hydrology']) + np.abs(test['Vertical_Distance_To_Hydrology'])

In [ ]:
def sum_soil_types(df):
    soil_cols = [col for col in df.columns if col.startswith('Soil_T')] 
    df['sum_soil_type'] = df.loc[:, soil_cols].sum(axis=1)
    return df

def summary_stats_features(df):
    df["mean"] = df.loc[:, df.columns].mean(axis = 1)
    df["min"] = df.loc[:, df.columns].min(axis = 1)
    df["max"] = df.loc[:, df.columns].max(axis = 1)
    
    return df
    
ss_types = FunctionTransformer(sum_soil_types)
ss_features = FunctionTransformer(summary_stats_features)

In [ ]:
X, y = train.loc[:, train.columns != 'Cover_Type'], train.Cover_Type

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 

In [ ]:
classifier = RandomForestClassifier(class_weight='balanced', n_jobs=-1)

clf = make_pipeline(ss_types, ss_features, StandardScaler(), RandomOverSampler(), RandomUnderSampler(), classifier)

In [ ]:
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
acc = balanced_accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Acc: {acc :.3f}\nF1 : {f1: .3f}')

In [ ]:
def submit(model):
    y_hat = clf.predict(test)
    sub = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
    sub['Cover_Type'] = y_hat
    sub.to_csv('submission.csv', index=False)
    
submit(clf)